In [209]:
#imports


from __future__ import print_function

import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import os

/home/bstc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'nbAgg' by the following code:
  File "/home/bstc/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/bstc/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/bstc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/bstc/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/bstc/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/

## Hyperparameters

In [210]:
batch_size = 1000   # num of training examples to train on in a batch
limit_timesteps = 20 # sequence length
num_hidden_layers = 300
state_dim = 3 # dimensions of the state variable passed as input to the LSTM model
future = 10
epochs = 100
csvname1 = 'states_2017-08-28-00.csv'
csvname2 = 'states_2017-10-02-00.csv'
filename = '8-28_10-02'

## Define PyTorch RNN Model

In [211]:
# Sequence model
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(state_dim, num_hidden_layers)
        self.lstm2 = nn.LSTMCell(num_hidden_layers, num_hidden_layers)
        self.linear = nn.Linear(num_hidden_layers, state_dim)

    def forward(self, input, future = 0):
        outputs = []
        
        h_t = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        
        # run 19 times
        # input_t.size = 5 x 5
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            input_t = input_t.squeeze(1)
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            if (future == 0):
                outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        print(outputs.size())
        return outputs

### Load data that will be used to train the model

In [212]:
# load data and make training set
data1 = pd.read_csv('cleaned/' + csvname1)
data2 = pd.read_csv('cleaned/' + csvname2)

data = data1.append(data2)

# # drop unneeded columns to form matrix to be trained on
# data_matr = data.drop(['NMAC_id', 'alert_1', 'alert_2', 'onground_1', 'onground_2'], axis=1).as_matrix().astype('float')

# restructure data
all_planes = []

count = 0
unique_icao = data.icao24.unique()
for i, name in enumerate(unique_icao):
    if count % 100 == 0: print(count)
    # only get batch size for now
    if count >= batch_size:
        break
    
    # get rows where icao == name (boolean vector)
    selector = data['icao24'] == name
    one_plane_df = data[selector]
    
    one_plane_df = one_plane_df.drop(['icao24','time','lat','lon','velocity','heading'], axis=1)
    one_plane_arr = one_plane_df.as_matrix()
    one_plane_arr = one_plane_arr[0:limit_timesteps]
    
    all_planes.append(one_plane_arr)
    
    count += 1

batch = np.asarray(all_planes) # convert to np matrix

0
100
200
300
400
500
600
700
800
900
1000


### Normalize data

In [213]:
# Normalize the x_displacement, y_displacement, geoaltitude and save to files
flat_batch = np.reshape(batch, (-1, state_dim))
means = np.mean(flat_batch, axis=0)
stdevs = np.std(flat_batch, axis=0)
batch -= means
batch /= stdevs

# save the means and stdevs to be denormalized in policy_generation
np.save(filename + "_model_means", means)
np.save(filename + "_model_stdevs", stdevs)

print(means)
print(stdevs)

[  6.21941504e+03  -7.16439857e-04   1.39462928e-03]
[  4.54121872e+03   1.25794934e-01   7.07828114e-02]


### Declare PyTorch input/target/test variables

In [214]:
input = Variable(torch.from_numpy(batch[100:,:-1,:]), requires_grad=False)
target = Variable(torch.from_numpy(batch[100:,1:,:]), requires_grad=False)
test_input = Variable(torch.from_numpy(batch[:100,:-future,:]), requires_grad=False)
test_target = Variable(torch.from_numpy(batch[:100,-future:,:]), requires_grad=False)
# data = torch.load('traindata.pt')
# input = Variable(torch.from_numpy(data[3:, :-1]), requires_grad=False)
# target = Variable(torch.from_numpy(data[3:, 1:]), requires_grad=False)
# test_input = Variable(torch.from_numpy(data[:3, :-1]), requires_grad=False)
# test_target = Variable(torch.from_numpy(data[:3, 1:]), requires_grad=False)
print(batch[0])
print(test_input.size())
print(test_target.size())
print(batch.shape)
# print(batch)

[[ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -5.06681263e-02  -3.35848153e+01   2.75911843e+01]
 [ -8.42273996e-02  -3.11501364e-01   1.04466509e-01]
 [ -1.36244273e-01  -8.06002143e-02   1.46826171e-02]
 [ -1.79871328e-01  -2.13266570e-01   6.51374158e-02]
 [ -2.10074674e-01  -1.57261129e-01   3.37442929e-02]
 [ -2.40278020e-01  -2.11256

## Build and train the PyTorch model

In [ ]:
# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()

# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

# begin to train
for i in range(epochs):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.data.numpy()[0])
        loss.backward()
        return loss
    optimizer.step(closure)
    
    # begin to predict
    pred = seq(test_input, future = future)
#     pred1 = seq(test_input[:1, :, :])
    loss = criterion(pred, test_target)
    print('test loss:', loss.data.numpy()[0])
    y = pred.data.numpy()
    if i % 10 == 0:
        torch.save(seq.state_dict(), filename + 'params_epoch{}.pt'.format(i))
print(pred)

#     # draw the result
#     plt.figure(figsize=(30,10))
#     plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
#     plt.xlabel('x', fontsize=20)
#     plt.ylabel('y', fontsize=20)
#     plt.xticks(fontsize=20)
#     plt.yticks(fontsize=20)
#     def draw(yi, color):
#         plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
#         plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
#     draw(y[0], 'r')
#     draw(y[1], 'g')
#     draw(y[2], 'b')
#     plt.savefig('predict%d.pdf'%i)
#     plt.close()


STEP:  0
torch.Size([900, 19, 3])
loss: 0.554976713847
torch.Size([900, 19, 3])
loss: 0.554575309925
torch.Size([900, 19, 3])
loss: 0.470054166476
torch.Size([900, 19, 3])
loss: 0.443507626263
torch.Size([900, 19, 3])
loss: 0.296718278642
torch.Size([900, 19, 3])
loss: 0.222176871921
torch.Size([900, 19, 3])
loss: 0.197878087808
torch.Size([900, 19, 3])
loss: 0.186889644133
torch.Size([900, 19, 3])
loss: 0.174428643346
torch.Size([900, 19, 3])
loss: 0.155549242653
torch.Size([900, 19, 3])
loss: 0.135023124453
torch.Size([900, 19, 3])
loss: 0.121147599202
torch.Size([900, 19, 3])
loss: 0.116928003832
torch.Size([900, 19, 3])
loss: 0.115515317732
torch.Size([900, 19, 3])
loss: 0.10854486055
torch.Size([900, 19, 3])
loss: 0.10012528256
torch.Size([900, 19, 3])
loss: 0.085138649981
torch.Size([900, 19, 3])
loss: 0.059650981652
torch.Size([900, 19, 3])
loss: 0.0679860390502
torch.Size([900, 19, 3])
loss: 0.0542186075088
torch.Size([100, 10, 3])
test loss: 2.26089332383
STEP:  1
torch.Size([

torch.Size([900, 19, 3])
loss: 0.0106899566383
torch.Size([900, 19, 3])
loss: 0.0106354082376
torch.Size([900, 19, 3])
loss: 0.0106063287961
torch.Size([900, 19, 3])
loss: 0.010594756634
torch.Size([900, 19, 3])
loss: 0.0105510346337
torch.Size([900, 19, 3])
loss: 0.0105388908972
torch.Size([900, 19, 3])
loss: 0.0105199403711
torch.Size([900, 19, 3])
loss: 0.0104776348654
torch.Size([900, 19, 3])
loss: 0.0104140529845
torch.Size([900, 19, 3])
loss: 0.0103581243909
torch.Size([900, 19, 3])
loss: 0.0103583569917
torch.Size([900, 19, 3])
loss: 0.0103129897018
torch.Size([900, 19, 3])
loss: 0.0103018399056
torch.Size([900, 19, 3])
loss: 0.0102893748944
torch.Size([900, 19, 3])
loss: 0.0102648487895
torch.Size([100, 10, 3])
test loss: 1.69624649308
STEP:  9
torch.Size([900, 19, 3])
loss: 0.0102382062708
torch.Size([900, 19, 3])
loss: 0.0101956255602
torch.Size([900, 19, 3])
loss: 0.0101663831785
torch.Size([900, 19, 3])
loss: 0.0101412896698
torch.Size([900, 19, 3])
loss: 0.0101241716667
to

torch.Size([900, 19, 3])
loss: 0.00788220404153
torch.Size([900, 19, 3])
loss: 0.00787069683802
torch.Size([900, 19, 3])
loss: 0.00785787981817
torch.Size([900, 19, 3])
loss: 0.00784673499953
torch.Size([900, 19, 3])
loss: 0.0078358594374
torch.Size([900, 19, 3])
loss: 0.00782701407287
torch.Size([900, 19, 3])
loss: 0.00782266754503
torch.Size([900, 19, 3])
loss: 0.00781868309771
torch.Size([900, 19, 3])
loss: 0.00781306254172
torch.Size([900, 19, 3])
loss: 0.00792778023232
torch.Size([900, 19, 3])
loss: 0.00780651256934
torch.Size([900, 19, 3])
loss: 0.007798860826
torch.Size([900, 19, 3])
loss: 0.00779397962796
torch.Size([100, 10, 3])
test loss: 1.68205935107
STEP:  17
torch.Size([900, 19, 3])
loss: 0.00778533947453
torch.Size([900, 19, 3])
loss: 0.0077771203467
torch.Size([900, 19, 3])
loss: 0.00776386448457
torch.Size([900, 19, 3])
loss: 0.00774158168357
torch.Size([900, 19, 3])
loss: 0.00772421632957
torch.Size([900, 19, 3])
loss: 0.00773462070611
torch.Size([900, 19, 3])
loss: 0

torch.Size([900, 19, 3])
loss: 0.00638445220795
torch.Size([900, 19, 3])
loss: 0.00638023844718
torch.Size([900, 19, 3])
loss: 0.00637639205659
torch.Size([900, 19, 3])
loss: 0.00637272725512
torch.Size([900, 19, 3])
loss: 0.00636933603017
torch.Size([900, 19, 3])
loss: 0.00646509059663
torch.Size([900, 19, 3])
loss: 0.0063668902831
torch.Size([900, 19, 3])
loss: 0.0063607399712
torch.Size([900, 19, 3])
loss: 0.00635774725034
torch.Size([900, 19, 3])
loss: 0.00635455268308
torch.Size([900, 19, 3])
loss: 0.00634712962653
torch.Size([100, 10, 3])
test loss: 1.65162645644
STEP:  25
torch.Size([900, 19, 3])
loss: 0.00634079797362
torch.Size([900, 19, 3])
loss: 0.00633576311756
torch.Size([900, 19, 3])
loss: 0.00645529109571
torch.Size([900, 19, 3])
loss: 0.00633537010495
torch.Size([900, 19, 3])
loss: 0.00633214617022
torch.Size([900, 19, 3])
loss: 0.00632761656099
torch.Size([900, 19, 3])
loss: 0.0063253448772
torch.Size([900, 19, 3])
loss: 0.00632243309145
torch.Size([900, 19, 3])
loss: 

In [ ]:
future = 10
test_pred = seq(test_input,future).data.squeeze(1).numpy()
test_pred_latlon = test_pred[:, :, -3:]
test_target_latlon = test_target[:, :, -3:]
i = 1
print('test_input', test_input[i])
print('test_target', test_target[i])
print('batch', batch[i])
for i in range(test_input.size(0)):
    latlon = test_input[i].data.numpy()[:, -3:]
    target_latlon = test_target_latlon[i].data.numpy()
    
    print(latlon)
    latlon = np.insert(latlon, latlon.shape[0], test_pred_latlon[i], axis=0)
    path = latlon.cumsum(axis=0)
    target_path = target_latlon.cumsum(axis=0) + path[-future-1]
    print(test_input[i])
    print("Test prediction:")
    print(test_pred[i])
    print("Path including prediction")
    print(path)
    pathx, pathy, pathz = np.split(path, 3, axis=1)
    pathz = pathz.flatten()
    
    target_pathx, target_pathy, target_pathz = np.split(target_path, 3, axis=1)
    target_pathz = target_pathz.flatten()
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.plot(pathx[0], pathy[0], pathz[0], 'b+')
    
    ax.plot(pathx[:-future], pathy[:-future], pathz[:-future], 'b.')
    ax.plot(target_pathx, target_pathy, target_pathz, 'g+')
    
#     print(pathx.shape)
    for j in range(1,future+1):
        plt.plot(pathx[-j], pathy[-j], pathz[-j], 'r.')
    
    plt.show()

## How to save and load models in pytorch

In [ ]:
# Self explanatory comment: Save only
torch.save(seq.state_dict(), filename + 'ACAS_theparameters.pt')

In [ ]:
# Self explanatory comment #2: Load only
seq1 = Sequence()
params = torch.load(filename + 'ACAS_theparameters.pt')
seq1.load_state_dict(params)
seq1.double()

## Making predictions and saving to csvs

- Denormalizes data according to model_stdevs.npy and model_means.npy
- Saves input sequences in "sequence/" folder, and their corresponding predictions in "prediction/" folder. 

In [ ]:
def denormalize(path):
    path *= np.load(filename + "_model_stdevs.npy")
    path += np.load(filename + "_model_means.npy")
    
    return path

# write to csv file
sequence_dir = 'sequences/'
pred_dir = 'predictions/'
if not os.path.exists(sequence_dir):
    os.makedirs(sequence_dir)
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

pred_df = pd.DataFrame(columns=['baroaltitude','lat','lon'])
future = 10
test_pred = seq(test_input,future).data.squeeze(1).numpy()
test_pred_latlon = test_pred[:, :, -3:]
for i in range(1,test_input.size(0)//10):
    latlon = test_input[i].data.numpy()[:, -3:]
    print(latlon)
    latlon = np.insert(latlon, latlon.shape[0], test_pred_latlon[i], axis=0)
    path = latlon.cumsum(axis=0)
    path = denormalize(path)
    pathz, pathx, pathy = np.split(path, 3, axis=1)
    pathz = pathz.flatten()
    
#     print("all data")
#     print(pathx)
#     print("just input data")
#     print(pathx[:-future])
    
    one_seq_df = pd.DataFrame(path[:-future], columns = ['baroaltitude', 'lat', 'lon'])
    one_pred_df = pd.DataFrame(path[future:], columns = ['baroaltitude', 'lat', 'lon'])
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_xlabel('lat')
    ax.set_ylabel('lon')
    ax.set_zlabel('baroaltitude')
    
    ax.plot(pathx[0], pathy[0], pathz[0], 'b+') # plot first sequence point
    ax.plot(pathx[:-future], pathy[:-future], pathz[:-future], 'b.') # plot other sequence points
    
    # plot predicted points
    for j in range(1,future+1):
        plt.plot(pathx[-j], pathy[-j], pathz[-j], 'ro')
    
    plt.show()
    
    #pred_df = pred_df.append(one_sequence_df, ignore_index=True)
    
    
    one_seq_df.to_csv(sequence_dir + filename + "_sequence_" + str(i) + ".csv", index=False)
    one_pred_df.to_csv(pred_dir + filename + "_sequence-pred_" + str(i) + ".csv", index=False)
    